# Vectorization Model of QuCT
**Author:** Siwei Tan  

**Date:** 7/4/2024

Based on paper "[QuCT: A Framework for Analyzing Quantum Circuit by Extracting Contextual and Topological Features][1]" (MICRO 2023)

[1]: https://dl.acm.org/doi/10.1145/3613424.3614274

In the current Noisy Intermediate-Scale Quantum era, quantum circuit analysis is an essential technique for designing high-performance quantum programs. Current analysis methods exhibit either accuracy limitations or high computational complexity for obtaining precise results. To reduce this tradeoff, we propose QuCT, a unified framework for extracting, analyzing, and optimizing quantum circuits. The main innovation of QuCT is to vectorize each gate with each element, quantitatively describing the degree of the interaction with neighboring gates. Extending from the vectorization model, we can develope multiple downstream models for fidelity prediction and unitary decomposition, etc. In this tutorial, we introduce the APIs of the vectorization model in QuCT.

In [1]:
%matplotlib inline

import os
os.chdir("..")
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

import random
import numpy as np

from janusq.analysis.vectorization import RandomwalkModel, extract_device
from janusq.data_objects.random_circuit import random_circuits, random_circuit
from janusq.data_objects.backend import GridBackend

## Vectorization Flow
Below is the workflow to vectorize a gate in the quantum circuit. The gate is vectorized by two steps. The first step runs random walks to extract circuit features in the neighbor of the gates. the second step use a table comparison to generate the gate vector.

<div style="text-align:center;">
    <img src="../picture/2_1_feature_extraction.jpg"  width="70%" height="70%">
</div>

## Random walk
We apply random walk to extract the topological and contextual information of gates in the quantum circuit. Here is a example of random walk.

In [2]:
from janusq.analysis.vectorization import walk_from_gate

# generate a backend and a circuit
backend = GridBackend(2, 2)
circuit = random_circuit(backend, 10, .5, False)
print(circuit)

# choose a target gate
gate = random.choice(circuit.gates)

# apply random walk
paths = walk_from_gate(circuit, gate, 4, 2, backend.adjlist)

print('target gate:', gate)
print('generate paths:', paths)

       ┌────────────────┐  ░                      ░ ┌─────────────────┐  ░ »
q_0: ──┤ U(3π/5,2π/5,π) ├──░──────────■───────────░─┤ U(3π/5,π/5,π/5) ├──░─»
     ┌─┴────────────────┴┐ ░        ┌─┴─┐         ░ └─────────────────┘  ░ »
q_1: ┤ U(7π/5,3π/5,9π/5) ├─░────────┤ X ├─────────░──────────────────────░─»
     ├───────────────────┤ ░ ┌──────┴───┴───────┐ ░ ┌──────────────────┐ ░ »
q_2: ┤ U(9π/5,4π/5,3π/5) ├─░─┤ U(8π/5,4π/5,π/5) ├─░─┤ U(7π/5,π/5,2π/5) ├─░─»
     ├───────────────────┤ ░ └──────────────────┘ ░ └──────────────────┘ ░ »
q_3: ┤ U(4π/5,2π/5,8π/5) ├─░──────────────────────░──────────────────────░─»
     └───────────────────┘ ░                      ░                      ░ »
«                          ░                     ░ 
«q_0: ─────────────────────░─────────────────────░─
«                          ░                     ░ 
«q_1: ─────────────────────░─────────────────────░─
«     ┌──────────────────┐ ░ ┌─────────────────┐ ░ 
«q_2: ┤ U(π/5,π/10,8π/5) ├─░─┤ U(2π,2π/5,2π/5) 

The above code generates the example path list of a random circuit. Each path is represented as `<gate_name>-<qubit_index>[, <dependency>, <gate_name>-<qubit_index>]`. For example, `u-2, parallel, cx-0-1, parallel, u-3` means that a U gate on qubit 2 is executed in parallel with a CX gate on qubits 0, 1 and a U gate on qubit 3. 

## Construction of Path Table

For a gate that requires vectorization, we compare it with a path table. The path table is off-line generated by applying random walks to a circuit dataset. To limits the size of the table, the table is usually hardware-specific.

In [3]:
# define the information of the quantum device
n_qubits = 6
backend = GridBackend(2, 3)

# generate a dataset including varous random circuits
circuit_dataset = random_circuits(backend, n_circuits=100, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

# apply random work to consturct the vectorization model with a path table
n_steps = 1
n_walks = 100
up_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend, decay= .5)
up_model.train(circuit_dataset, multi_process=False)
print('length of the path table is', len(up_model.pathtable))

100%|██████████| 102/102 [00:00<00:00, 214.66it/s]


length of the path table is 445


# Gate Vectorization

As mentioned above, the vectorization of a gate is performed by comparing the generated paths with a path table. In JanusQ, we provide an API for it.

In [4]:
# generate a circuit
circuit = random_circuit(backend, 10, .5, False)

# choose a target gate
gate = random.choice(circuit.gates)

# vectorization
vec = up_model.vectorize(circuit, [gate])[0]
print('vector is', vec)

vector is [0.5 0.5 0.5 0.5 0.  0.  0.5 0.  0.5 1.  0.  0.  0.5 0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.
 0. ]


The indexes of the non-zero elements in the vector is same to the indexes of the generated paths in the path table, which is verified by following codes.

In [5]:
# validate the vectorization
indexes = np.argwhere(vec > 0).flatten()
generated_paths = walk_from_gate(circuit, gate, 100, 1, backend.adjlist)
device = extract_device(gate)
print(list(indexes), '=', up_model.indices_of_paths(device, generated_paths))

[0, 1, 2, 3, 6, 8, 9, 12, 22, 28] = [0, 28, 1, 2, 3, 6, 22, 8, 9, 12]


## Sub-circuit Reconstruction
The vectorization of QuCT also allows the reconstruction of the sub-circuit around the gate by its vector.

In [6]:
# reconstruct the circuit from the vector
circuit = up_model.reconstruct(device, vec)
print(circuit)

      ┌──────────────────────────┐ ░                                   ░      »
q_0: ─┤ U(5.3992,5.2086,0.46458) ├─░───────────────────────────────■───░──────»
      └┬───────────────────────┬─┘ ░                               │   ░      »
q_1: ──┤ U(4.802,5.3146,3.443) ├───░───────────────────────────────┼───░──────»
      ┌┴───────────────────────┴─┐ ░ ┌──────────────────────────┐  │   ░      »
q_2: ─┤ U(0.67875,3.1333,2.3294) ├─░─┤ U(5.2502,1.6739,0.52743) ├──┼───░───■──»
     ┌┴──────────────────────────┤ ░ └──────────────────────────┘┌─┴─┐ ░   │  »
q_3: ┤ U(0.016824,4.0638,2.9548) ├─░─────────────────────────────┤ X ├─░───┼──»
     └┬─────────────────────────┬┘ ░                             └───┘ ░   │  »
q_4: ─┤ U(5.2491,1.8757,3.8353) ├──░──────────────■────────────────────░───┼──»
      ├─────────────────────────┤  ░            ┌─┴─┐                  ░ ┌─┴─┐»
q_5: ─┤ U(5.8717,1.4803,1.3123) ├──░────────────┤ X ├──────────────────░─┤ X ├»
      └─────────────────────────┘  ░    